In [1]:
##### DOWNLOAD DATASET #####
from IPython.display import clear_output
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d gsimonx37/letterboxd
!unzip /content/letterboxd.zip -d /content/letterbox/
clear_output()
############################

KeyboardInterrupt: 

In [1]:
########### SPARK CONTEXT #####################
import pandas as pd
import itertools

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = spark.sparkContext
###############################################

In [2]:
############## BASKET CREATION ################
#TODO do it using SPARK directly -> cars = spark.read.csv('cars.csv', header=True, sep=";")
actors = pd.read_csv("letterbox/actors.csv")
actors = actors.sample(100000)
baskets = actors.groupby("id")["name"].apply(list)

print("number of baskets: " + str(len(baskets)))
print("biggest basket: " + str(baskets.map(len).max()))
print(baskets)
baskets_RDD = sc.parallelize(baskets).cache()
##############################################

number of baskets: 85542
biggest basket: 11
id
1000001         [Oliver Chapman, Joelle Dyson, Lewis Easter]
1000002                                    [Choi Jeong-hyun]
1000004                                      [Leonard Termo]
1000006    [Leigh Gill, Isabella Ferreira, Ben Warheit, R...
1000010                                    [Susan Griffiths]
                                 ...                        
1896087                      [Prince Kuhlmann, Meike Droste]
1896093                                        [Lee Francis]
1896140                                       [Smith Mathew]
1896274                                   [Claire Elizabeth]
1896351                                  [Christine McMahon]
Name: name, Length: 85542, dtype: object


In [3]:
baskets_RDD.getNumPartitions()

2

In [17]:
## APRIORI
def apriori(chunk, s, tot_bsk):
  """

  Args:
    chunk:
    s:
    tot_bsk:

  Yields:

  """

  n_bsk = 0 # number of baskets in chunk used to calculate support
#  trsl = dict() # using a translation could be better for the combination step
  count = dict() # should be relatively efficient since is a hash table

#
#  def translate(e, dic):
#    try:
#      return dic[e]
#    except KeyError:
#      dic[e] = len(dic)+1
#      return dic[e]
  def incr(e,dic):
    try:
      dic[e] += 1;
    except KeyError:
      dic[e] = 1

  chunk = list(chunk) # to avoid consuming after first pass
  ### first pass
  for bsk in chunk:
    n_bsk += 1;
    #elements = [translate(x,trsl) for x in bsk]
    for b in bsk:
      incr(b,count)

  ### filter
  to_del = []
  for c in count:
    if count[c] < (n_bsk * s)/tot_bsk:
      to_del.append(c)
  for t in to_del:
    del count[t]



  frequent = list(count.keys())
  #yield (len(frequent),1)

  frequent.sort()
  cmb = list(itertools.combinations(frequent,2))
  #yield (len(cmb),1)

  count_2 = dict()
  aux = 0
  ### second pass
  for bsk in chunk:
    for pair in cmb: #this already solve the ordering problem
      aux += 1
      if pair[0] in bsk and pair[1] in bsk:
        incr(tuple(pair), count_2)

  to_del = []
  for c in count_2:
    if count_2[c] < (s * n_bsk)/tot_bsk:
      to_del.append(c)
  for t in to_del:
    del count_2[t]
  ### out
  #yield (aux,1)
  for c in count_2:
    yield (c,1)


In [18]:

def first_red(key,value):
  yield key
def second_map(candidate, chunk):
  yield (1,1)
def second_red(key,value):
  yield key

chunk = 1; #TMP to remove
s =  10; #support threshold
tot = len(baskets)

baskets_RDD.mapPartitions(lambda x: apriori(x,s,tot)).reduceByKey(lambda a,b: a).collect()


[]

In [ ]:
print(min(actors["id"]))
print(max(actors["id"]))
print(len(actors))
len(set(actors["name"]))

In [ ]:
print(len(baskets))

In [ ]:
len(actors[actors['name'] == 'Bess Flowers'])

In [ ]:
#x = [1,2,3,4,5,6,9]
x = ["abaco","banana", "zorro","abba","abecedario"]
for i in itertools.combinations(x,2):
  print(i)